In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Movie, Actor, Director, Genre, create_db
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
engine = create_engine('sqlite:///myblog.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()


def create_cosine_matrix(arr):
    cm = CountVectorizer().fit_transform(arr)
    cs = cosine_similarity(cm)
    return cs


def get_similars(cosine_matrix, row, num):
    scores = {i: s for i, s in enumerate(cosine_matrix[row]) if row != i}
    scores = scores.items()
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    similars = [i[0] for i in scores[:num]]
    return similars


df = pd.read_csv("movies.csv")

df = df[["title", "genre", "imdb_rating", "director", "cast"]]
df

,title,genre,imdb_rating,director,cast
0,The Shawshank Redemption,Drama,9.3,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,The Godfather,"Crime, Drama",9.2,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,The Dark Knight,"Action, Crime, Drama",9.0,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M..."
3,The Godfather: Part II,"Crime, Drama",9.0,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian..."
4,12 Angry Men,"Crime, Drama",9.0,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ..."
...,...,...,...,...,...
95,Inglourious Basterds,"Adventure, Drama, War",8.3,Quentin Tarantino,"Brad Pitt, Diane Kruger, Eli Roth, Mélanie Lau..."
96,Eternal Sunshine of the Spotless Mind,"Drama, Romance, Sci-Fi",8.3,Michel Gondry,"Jim Carrey, Kate Winslet, Tom Wilkinson, Gerry..."
97,Amélie,"Comedy, Romance",8.3,Jean-Pierre Jeunet,"Audrey Tautou, Mathieu Kassovitz, Rufus, Lorel..."
98,Snatch,"Comedy, Crime",8.3,Guy Ritchie,"Jason Statham, Brad Pitt, Benicio Del Toro, De..."


In [51]:
seens = []
def combine(row):
    return " ".join([str(c) for c in row])

for i in range(len(df.head(10))):
    seens.append(combine(df.iloc[i]))
seens = " ".join(seens)
print(seens)
unseens = []
u = df[10:]
for i in range(len(u)):
    unseens.append(combine(u.iloc[i]))

m = create_cosine_matrix([seens, *unseens])
similars = get_similars(m, 0, 20)
for i in similars:
    print(f"{u['title'].iloc[i]}")

The Shawshank Redemption Drama 9.3 Frank Darabont Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler The Godfather Crime, Drama 9.2 Francis Ford Coppola Marlon Brando, Al Pacino, James Caan, Diane Keaton The Dark Knight Action, Crime, Drama 9.0 Christopher Nolan Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine The Godfather: Part II Crime, Drama 9.0 Francis Ford Coppola Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton 12 Angry Men Crime, Drama 9.0 Sidney Lumet Henry Fonda, Lee J. Cobb, Martin Balsam, John Fiedler The Lord of the Rings: The Return of the King Action, Adventure, Drama 8.9 Peter Jackson Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom Pulp Fiction Crime, Drama 8.9 Quentin Tarantino John Travolta, Uma Thurman, Samuel L. Jackson, Bruce Willis Schindler's List Biography, Drama, History 8.9 Steven Spielberg Liam Neeson, Ralph Fiennes, Ben Kingsley, Caroline Goodall Inception Action, Adventure, Sci-Fi 8.8 Christopher Nolan Leonardo DiCaprio, Josep